In [ ]:

import pandas as pd


countries = ["uk","italy","france","germany"]
products = ["cosmetics","fashion","watches","jewelry"]

tables = {
    "cosmetics": {"uk": [], "italy": [], "france": [], "germany": []},
    "fashion": {"uk": [], "italy": [], "france": [], "germany": []},
    "watches": {"uk": [], "italy": [], "france": [], "germany": []},
    "jewelry": {"uk": [], "italy": [], "france": [], "germany": []}
}

# we loaded data for all four products just like this. here we are just showing one for cosmetics.

data = pd.read_csv("uk_cosmetics.csv", encoding="latin-1", header=None)


headings = {
    "gender": pd.DataFrame(
        [["answer"] + data.iloc[2, 2:7].dropna().tolist() + ["country"]]
    ),

    "age": pd.DataFrame(
        [["answer"] + data.iloc[2, 7:15].dropna().tolist() + ["country"]]
    ),

    "income": pd.DataFrame(
        [["answer"] + data.iloc[2, 15:25].dropna().tolist() + ["country"]]
    )
}

# Loop through each product and each country and extract questions and tables.
# Add country column and separate the tables by demographic.
# At the end, add each question and (new) table to the nested dictionary "new_tables".

for product in products:


  for country in countries:


    data = pd.read_csv(f"{country}_{product}.csv", encoding="latin-1", header=None)

    survey_positions = data[data.apply(lambda x: x.astype(str).str.contains("Survey Name:").any(), axis=1)].index
    base_positions = data[data.apply(lambda x: x.astype(str).str.contains("Base").any(), axis=1)].index[::2]

    question_type = list(data[data.apply(lambda x: x.astype(str).str.contains("Question Type").any(), axis=1)].index)

    question_type_count = 0

    for start, end in zip(base_positions,survey_positions):


        if data.iloc[question_type[question_type_count]][1] == "Single Pick":

          question = data.iloc[start-3][0]

          table = data.iloc[start: end].fillna(" ")

          cols_to_drop = table.iloc[0] == "in %"

          table = (table.loc[:, ~cols_to_drop]).iloc[1:]

          gender_table = pd.concat([table.iloc[:, [0]], table.iloc[:,2:4]], axis=1)
          gender_table["country"] = country
          age_table = pd.concat([table.iloc[:, [0]], table.iloc[:,4:8]], axis=1)
          age_table["country"] = country
          income_table = pd.concat([table.iloc[:, [0]], table.iloc[:,8:13]], axis=1)
          income_table["country"] = country

          tables[product][country].append({
              "question": question,
              "table": {"gender": gender_table, "age": age_table, "income": income_table}
          })

        question_type_count += 1



new_tables = {"cosmetics":[],"fashion":[],"watches":[],"jewelry":[]} # might not need.




In [ ]:
# At this step, we would have empty product uploaded, to which we upload the new contingency tables, using our nested dictionary.

for product in new_tables:

    with open(f"{product}.csv", "w", encoding="utf-8") as file:
      file.write("\n")


      uk_dict = tables[product]["uk"]
      italy_dict = tables[product]["italy"]
      france_dict = tables[product]["france"]
      germany_dict = tables[product]["germany"]

      for count in range(0,len(uk_dict)):

        question = uk_dict[count]["question"]
        file.write("QUESTION: "+ question + '\n')

        for covariate in headings:

          headings[covariate].to_csv(file, header=False, index=False)
          uk_dict[count]["table"][covariate].to_csv(file, header=False, index=False)
          germany_dict[count]["table"][covariate].to_csv(file, header=False, index=False)
          italy_dict[count]["table"][covariate].to_csv(file, header=False, index=False)
          france_dict[count]["table"][covariate].to_csv(file, header=False, index=False)
          file.write('\n\n')

      file.write("\n\n\n\n")

